In [13]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import cython
import sys
#!{sys.executable} -m pip install sklearn
import sklearn



import os
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import time
import numpy as np
import tensorflow as tf

stops = None
stemmer = PorterStemmer()

translator = str.maketrans('', '', string.punctuation)
word2VecDirectory_Input = os.getcwd() + "/Word2VecData_Input/" 
word2VecDirectory_Output = os.getcwd() + "/Word2VecData_Output/" 

df_train = pd.read_csv('train.csv')
modelfileName = "CS256"

In [14]:
def getStopWords():
    global stops
    stops = set(stopwords.words("english"))
    stops.add("what")

In [15]:
def preprocessString(CurrentString):
    
    if CurrentString is None: return None
    CurrentString = str(CurrentString)
    CurrentString = CurrentString.translate(translator)
    result = ""
    CurrentString = CurrentString.split()
    for word in CurrentString:
        if word is None or word == "": continue
        try:
            word = stemmer.stem(word.lower())
        except:
            continue
            
        if word.lower() not in stops:
            result += word + " "
    return result

def getModelData():
    import time
    #start = time.time()
    global combinedWordSet
    global translator
    global stemmer
    global stops
    
    getStopWords()

    filenames = ['train.csv']
    columnNames = ['question1', 'question2']
    for i in range(len(filenames)):
        df = pd.read_csv(filenames[i])
        for j in range(len(columnNames)):
            fOutput = open(word2VecDirectory_Input + columnNames[j] + ".txt", "w")
            for question in df[columnNames[j]]:
                result = preprocessString(question)
                if result is None: continue
                fOutput.write(result + "\n")
            fOutput.close()
        
        fOutput = open(word2VecDirectory_Output + "is_duplicate.txt", "w")
        for word in df['is_duplicate']:
            fOutput.write(str(word) + "\n")
        fOutput.close()
            
    end = time.time()
    print("Processing Ends")
    #print("Total time needed -> ", (end - start))

In [16]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(word2VecDirectory_Input)
    print('data direcrtory-->',dataDirec)
    model = Word2Vec(dataDirec, size=400, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [17]:
# This step takes a bit of time to process
getModelData()
createWord2VecModels()

Processing Ends
data direcrtory--> <__main__.IteratingClass object at 0x12ba8eb38>


Training Successful for Word2Vec Model!!!
Total Time for Word2Vec model ->  44.86117887496948


In [18]:
def getCosineAngle(a, b):

    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)


In [19]:


#x_train = None
def getNumpyArray():
    start = time.time()
    
    global X
    global Y
    global dirNameInput
    X = None
    Y = None
    global question1
    global question2
    global length
    global cosineDistance
    
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Processing started ")
    question1 = []
    with open(word2VecDirectory_Input + "question1.txt", "r") as fQuestion1:
        question1 = fQuestion1.readlines()
    question1 = [x.strip() for x in question1] 
    
    question2 = []
    with open(word2VecDirectory_Input + "question2.txt", "r") as fQuestion2:
        question2 = fQuestion2.readlines()
    question2 = [x.strip() for x in question2] 
    
    similar = []
    with open(word2VecDirectory_Output + "is_duplicate.txt", "r") as fsimilar:
        similar = fsimilar.readlines()
    similar = [x.strip() for x in similar] 
    
    print(len(similar))
    print(len(question1))
    print(len(question2))

    length = len(similar) if len(similar) < len(question1) else len(question1)
    length = length if length < len(question2) else len(question2)
    
    X_sub = []
    Y_sub = []
    cosineDistance= []
    wordMoverDistance = []
    averageDistance = []
    
    for i in range(length):
        
        q1Vec = []
        
        parts = question1[i].split()
        if len(parts) < 1: continue 
        currV = Word2Vec_model.wv[parts[0]]
        currV.setflags(write=1)
        for i in range(len(parts)):
            currV += Word2Vec_model.wv[parts[i]]
        currV /= len(parts)
        q1Vec = np.array(currV)
        
        parts = question2[i].split()
        if len(parts) < 1: continue
        currV = Word2Vec_model.wv[parts[0]]
        currV.setflags(write=1)
        for i in range(len(parts)):
            currV += Word2Vec_model.wv[parts[i]]
        currV /= len(parts)
        q2Vec = np.array(currV)
        
        averageDistance.append(np.linalg.norm(q1Vec - q2Vec))
        
        distance = Word2Vec_model.wv.wmdistance(question1[i].split(), question2[i].split())
        wordMoverDistance.append(distance)
        
        cosineSimilarity = getCosineAngle(q1Vec, q2Vec)
        cosineDistance.append(cosineSimilarity)
        
        currV = int(similar[i])
        
        Y_sub.append(np.array(currV))
        X_sub.append(np.concatenate((q1Vec, q2Vec), axis=0))
        
    
    X = np.array(X_sub)
    Y = np.array(Y_sub)
    print(X.shape)    
    print(Y.shape)   
    print(len(X))
   
    print('WMD Distance List-->',len(wordMoverDistance))
    print('Cosine Distance-->',len(cosineDistance))
    print('Average distance-->',len(averageDistance))
    

    #write Cosine Distance , Word Mover's Distance to CSV   
    rows = zip(wordMoverDistance,cosineDistance,averageDistance)
    with open("FeatureExtracted.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(['WMD', 'CD', 'Avg'])
        for row in rows:
            writer.writerow(row)
    
        
    #print("total time -> ", end -start)

In [ ]:
# This step too takes a bit of time to process
getNumpyArray()

Processing started 
404290
404290
404290


In [10]:
def createDNNModel():
    global model     
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    

    

    

In [11]:
createDNNModel()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.30, random_state=1)
print('Head of traing-->',x_train.head())


In [12]:
def executeModel(eph):
    start = time.time()
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=eph)
    end = time.time()
    print("Total Time -> ", (end - start))
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(val_loss)
    print(val_acc)



In [13]:
# This step takes time too
start = time.time()
executeModel(10)
end = time.time()
print("Total Time -->", end - start)

Epoch 1/10
282957/282957 [==============================] - 83s 293us/step - loss: 0.2718 - acc: 0.9829
Epoch 2/10
282957/282957 [==============================] - 85s 299us/step - loss: 0.2770 - acc: 0.9827
Epoch 3/10
 60672/282957 [=====>........................] - ETA: 1:08 - loss: 0.2704 - acc: 0.9832

KeyboardInterrupt: 